In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests

def get_percentage(x):
    return (np.exp(x) - 1) * 100
SPECIAL_COUNTRIES = ["ISR", "AUS", "CAN", "USA"]
def granger_w_zero_lag(df, maxlag, **kwargs):
    df_copy = df.copy()
    df_copy.iloc[:, 1] = df_copy.iloc[:, 1].shift(-1)
    df_copy = df_copy.fillna(0)
    return grangercausalitytests(df_copy, maxlag, **kwargs)

In [ ]:
# download vax dataset at "https://github.com/owid/covid-19-data/raw/master/public/data/vaccinations/vaccinations.csv"
# And put it in "../data/owid_vaccinations.csv"

# nuts = pd.read_csv("../data/NUTS2021.csv")
vax = pd.read_csv("../data/owid_vaccinations.csv")
vax["date"] = pd.to_datetime(vax["date"])
vax.head()
vax["country"] = vax["iso_code"]
vax.loc[vax["iso_code"] == "DEU", "country"] = "DEUTNP"
vax.loc[vax["iso_code"] == "FRA", "country"] = "FRATNP"
vax.loc[vax["iso_code"] == "NZL", "country"] = "NZL_NP"
vax.loc[vax["iso_code"] == "OWID_SCT", "country"] = "GBR_SCO"

for country in ["SWE", "RUS", "SVK", "FIN", "OWID_ENG", "OWID_WLS", "NLD", "PRT"]:
# for country in vax.iso_code.unique():
    vax.loc[vax["iso_code"] == country, ["people_vaccinated"]] = vax[vax["iso_code"] == country]["people_vaccinated"].interpolate(method="ffill")
    vax.loc[vax["iso_code"] == country, ["total_boosters"]] = vax[vax["iso_code"] == country]["total_boosters"].fillna(method="ffill")
    vax.loc[vax["iso_code"] == country, ["people_fully_vaccinated"]] = vax[vax["iso_code"] == country]["people_fully_vaccinated"].fillna(method="ffill")
    vax.loc[vax["iso_code"] == country, ["total_vaccinations"]] = vax[vax["iso_code"] == country]["total_vaccinations"].fillna(method="ffill")


df_GBRTENW = vax[vax["iso_code"] == "OWID_ENG"].copy()
df_GBRTENW.index = pd.to_datetime(df_GBRTENW.date)

df_GBRTENW["country"] = "GBRTENW"

for col in ["people_vaccinated", "total_boosters", "people_fully_vaccinated", "total_vaccinations"]:
    other_df = vax[vax["iso_code"] == "OWID_WLS"]
    other_df.index = pd.to_datetime(other_df.date)
    df_GBRTENW.loc[other_df.index, col] += other_df[col].values
# df_GBRTENW.loc[:, "people_fully_vaccinated"] = df_GBRTENW.loc[:, "total_vaccinations"]

vax = pd.concat([vax, df_GBRTENW])

In [ ]:
# load data from the "all_countries" notebook computing excess mortality
all_ages_df = pd.read_csv("../data/mortality_all_ages_normalized.csv")
all_ages_df["date"] = pd.to_datetime(all_ages_df.date)
all_ages_df.head()


In [ ]:
all_ages_df.country.unique()

In [ ]:
vax.iso_code.unique()

In [ ]:
# Calculate weekly doses administered

combo = pd.merge(vax, all_ages_df, on=["country", "date"], how='right')

dfs = []
for group, df in combo.groupby(by=["country", "sex", "age_group"]):
#     if group[0] != "SVK" or group[1] != "b" or group[2] != "adult":
#         continue
    weekly_vax = df.people_vaccinated.values[1:] - df.people_vaccinated.values[:-1]
    weekly_vax[weekly_vax == 0] = np.nan
    weekly_fully_vax = df.people_fully_vaccinated.values[1:] - df.people_fully_vaccinated.values[:-1]
    weekly_fully_vax[weekly_fully_vax == 0] = np.nan
    total_boosters_vax = df.total_boosters.values[1:] - df.total_boosters.values[:-1]
    total_boosters_vax[total_boosters_vax == 0] = np.nan

    df.loc[:,"weekly_vaccinated"] = np.insert(weekly_vax, 0, df.people_vaccinated.values[0])
    df.loc[:,"weekly_fully_vax"] = np.insert(weekly_fully_vax, 0, df.people_fully_vaccinated.values[0])
    df.loc[:,"total_boosters_vax"] = np.insert(total_boosters_vax, 0, df.total_boosters.values[0])
    for col in ["weekly_vaccinated", "weekly_fully_vax", "total_boosters_vax"]:
        if group[0] == "ISL":
            pass
#             df.loc[:, col] = df.loc[:, col].fillna(method="fill")
        else:
            df.loc[:, col] = df.loc[:, col].interpolate()
    df.loc[df["weekly_vaccinated"].isna(), "weekly_vaccinated"] = 0
    df.loc[df["weekly_fully_vax"].isna(), "weekly_fully_vax"] = 0
    df.loc[df["total_boosters_vax"].isna(), "total_boosters_vax"] = 0
    df["weekly_total_vax"] = df.loc[:,"weekly_vaccinated"] + df.loc[:,"weekly_fully_vax"] + df.loc[:,"total_boosters_vax"]
    df["centered_per"] = get_percentage(df["centered"].values)
    df["centered_per_low"] = get_percentage(df["centered_low"].values)
    df["centered_per_high"] = get_percentage(df["centered_high"].values)
    df.loc[:, "location"] = df["location"].fillna(method="ffill").fillna(method="bfill")
#     if group[0] == "ISL" and group[1] == "b" and group[2] == "adult":
#         break
    
    dfs.append(df)
combo = pd.concat(dfs)
# df.index = pd.to_datetime(df.date)
# df = df[~df.iso_code.isnull()]
# df.loc[:, col].interpolate()["2021-10":]

In [ ]:
# Calculate everate excess mortality post covid
countries = all_ages_df.country.unique()
pd.options.mode.chained_assignment = None  # default='warn'
avg_mortality_percent = {}
specific = {}
for country in countries:
    specific[country] = combo[(combo.country == country) & 
                (combo.date.dt.year.isin([2020, 2021, 2022])) & 
                (combo.age_group == "high") &
               (combo.sex == "b")]

    avg_mortality_percent[country] = specific[country].centered_per.mean()


In [ ]:
# PLOTS
n_cols = 2
n_rows = int(np.ceil(n_countries / n_cols))
age_string = {"high": "65+", "mid": "15-64", "adult": "15+"}

n_countries = len(special_countries)
for group, age_df in combo.groupby(by=["age_group", "sex"]):
    age_group, sex = group
    if age_group != "adult" or sex != "b":
        continue
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(14,5))
    i = 0
    for country, df in age_df.groupby(by=["country"]):
        if country not in SPECIAL_COUNTRIES:
            continue
        j = i % n_cols
        k = i // n_cols
        axs[k, j].plot(df.date, df.centered_per, color="black", linewidth=2, label="mortality [%]")
        axs[k, j].plot(df.date, df.centered_per_low, color="black", linestyle='--')
        axs[k, j].plot(df.date, df.centered_per_high, color="black", linestyle='--')
        axs[k, j].set_title(df.loc[:, "location"].values[0], fontsize=20)
        axs[k, j].set_xlim([np.datetime64("2020-01-01"), np.datetime64("2023-01-01")])

        ax2 = axs[k, j].twinx()    
        any_dose = df.weekly_vaccinated + df.weekly_fully_vax + df.total_boosters_vax
        ax2.plot(df.date, any_dose, label="# doses", color="red", alpha=0.7)
        if j == n_cols - 1:
            ax2.set_ylabel("# Doses")
        if k == 0:
            axs[k, j].legend(loc="upper left")
            ax2.legend(loc="upper right")
        i += 1

    fig.add_subplot(111, frameon=False)
    plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
    plt.grid(False)
    plt.ylabel(f"Excess mortality, age {age_string[age_group]} [% from baseline]")
    fig.tight_layout()
    plt.savefig(f"../figures/special_countries_mortality_vs_vax_{age_group}_age_{sex}_sex.jpg", dpi=100)


In [ ]:

n_cols = 2
n_rows = int(np.ceil(n_countries / n_cols))
age_string = {"high": "65+", "mid": "15-64", "adult": "15+"}

n_countries = len(special_countries)
for group, age_df in combo.groupby(by=["age_group", "sex"]):
    age_group, sex = group
    if age_group != "adult" or sex != "b":
        continue
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(14,5))
    i = 0
    for country, df in age_df.groupby(by=["country"]):
        if country not in SPECIAL_COUNTRIES:
            continue
        j = i % n_cols
        k = i // n_cols
        axs[k, j].plot(df.date, df.centered_per, color="black", linewidth=2, label="mortality [%]")
        axs[k, j].plot(df.date, df.centered_per_low, color="black", linestyle='--')
        axs[k, j].plot(df.date, df.centered_per_high, color="black", linestyle='--')
        axs[k, j].set_title(df.loc[:, "location"].values[0], fontsize=20)
        axs[k, j].set_xlim([np.datetime64("2020-01-01"), np.datetime64("2023-01-01")])

        ax2 = axs[k, j].twinx()    
        any_dose = df.total_boosters_vax
        ax2.plot(df.date, any_dose, label="# doses", color="red", alpha=0.7)
        if j == n_cols - 1:
            ax2.set_ylabel("# Doses")
        if k == 0:
            axs[k, j].legend(loc="upper left")
            ax2.legend(loc="upper right")
        i += 1

    fig.add_subplot(111, frameon=False)
    plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
    plt.grid(False)
    plt.ylabel(f"Excess mortality, age {age_string[age_group]} [% from baseline]")
    fig.tight_layout()
    plt.savefig(f"../figures/special_countries_mortality_vs_booster_{age_group}_age_{sex}_sex.jpg", dpi=100)


In [ ]:
n_countries = len(SPECIAL_COUNTRIES)
n_cols = 2
n_rows = int(np.ceil(n_countries / n_cols))
age_string = {"high": "65+", "mid": "15-64", "adult": "15+"}

for age_group, age_df in combo.groupby(by=["age_group"]):
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(14,5))
    i = 0
    for gdf in age_df.groupby(by=["country"]):
        country, df = gdf
        if country not in SPECIAL_COUNTRIES:
            continue
        df = df[(df["sex"] == "b")]
        df.index = pd.to_datetime(df.date)

        j = i % n_cols
        k = i // n_cols
        axs[k, j].plot(df.date, df["fit.yearly"].values, color="gold", linewidth=2, label="trend")
        axs[k, j].plot(df.date, df["fit.yearly"].values + df["fit.weekly"].values, color="gold", linewidth=2)
        axs[k, j].plot(df.date, df["fit.yearly"].values + df["fit.weekly"].values + df["centered"].values, color="black", linewidth=1, label="mortality")
        axs[k, j].plot(df.date, df["fit.yearly"].values + df["fit.weekly"].values + df["centered_low"].values, color="black", linestyle='--', linewidth=0.5, label="95 confidence int.")
        axs[k, j].plot(df.date, df["fit.yearly"].values + df["fit.weekly"].values + df["centered_high"].values, color="black", linestyle='--', linewidth=0.5)
        axs[k, j].set_title(df.loc[:, "location"].values[0], fontsize=20)
    #     axs[k, j].set_xlim([np.datetime64("2017-01-01"), np.datetime64("2023-01-01")])

        if j == 0 and k % n_cols == 0:
            pass
#             axs[k, j].set_ylabel(f"Mortality, age {age_string[age_group]} [# deaths/100k/week]")
        if k == 1 and j == 1:
            axs[k, j].legend(loc="upper left")
#             ax2.legend(loc="upper right")
        i += 1
    fig.add_subplot(111, frameon=False)
    plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
    plt.grid(False)
    plt.ylabel(f"Mortality, age {age_string[age_group]} [# deaths/100k/week]")
    fig.tight_layout()
    plt.savefig(f"../figures/special_countries_mortality_w_trend_b_gender_{age_group}_age.jpg", dpi=100)


In [ ]:
country

In [ ]:
# Shift vax with max autocorrelation
# df = combo[(combo.age_group == "adult") & (combo.sex == "b") & (combo.country == "USA")]
lags = {}
max_lags = 11
vax_type = "total_boosters_vax"
for country, df in combo.groupby(by=["country"]):
    if country in ["RUS", "GBR_NIR", "HRV", "ISL"]:
        lags[country] = (0, np.nan)
        continue
    df = df[(df.age_group == "adult") & (df.sex == "b")]
    first_dose = np.arange(len(df))[df[vax_type] > 100][0]
    last_dose = np.arange(len(df))[df[vax_type] > 100][-1]

    h = plt.xcorr(df.centered_per, df[vax_type], maxlags=13)
    pos_ind = h[0] >= 0
    argmax = np.argmax(h[1][pos_ind])
    if country == "NOR":
        print(argmax)
    if h[0][pos_ind][argmax] == max_lags:
        lags[country] = (0, np.nan)
    else:
        lags[country] = (h[0][pos_ind][argmax], np.round(h[1][pos_ind][argmax], 2))

In [ ]:
# Plot Norway's data shifted 10 weeks
country = "NOR"
lag = lags[country][0]
df = combo[(combo.age_group == "adult") & (combo.sex == "b") & (combo.country == country)]
fig, ax = plt.subplots(1,1, figsize=(12,4))
plt.plot(df.date, df.centered_per, color="black", linewidth=2, label="mortality [%]")
plt.plot(df.date, df.centered_per_low, color="black", linestyle='--', label="95% confidence band")
plt.plot(df.date, df.centered_per_high, color="black", linestyle='--')
# plt.title(df.loc[:, "location"].values[0], fontsize=20)
plt.xlim([np.datetime64("2020-01-01"), np.datetime64("2023-01-01")])
plt.ylabel(f"Excess mortality, age {age_string[age_group]} [% from baseline]")
plt.legend(loc="upper left")
# plt.grid()
ax2 = plt.twinx()    
any_dose = df.weekly_total_vax
first_dose = np.arange(len(any_dose))[any_dose > 100][0]
last_dose = np.arange(len(any_dose))[any_dose > 100][-1]
ax2.plot(df.date, any_dose.values, label="# doses", color="red", alpha=0.2)
if lag != 0:
    ax2.plot(df.date[lag:], any_dose.values[:-lag], label=f"# doses shifted {lag} weeks", color="blue", alpha=0.7)
else:
    ax2.plot(df.date, any_dose, label="# doses", color="red", alpha=0.7)
# h = granger_w_zero_lag(df.loc[:, ["centered_per", "total_boosters_vax"]].iloc[first_dose:last_dose,:], lag + 2, verbose=True)
title = f"{df.loc[:, 'location'].values[0]} - vaccination shifted {lags[country][0]} weeks"
plt.title(title, fontsize=20)
ax2.set_ylabel("# Doses")

ax2.legend(loc="upper right")
plt.savefig(f"../figures/norway_mortality_vs_vaccine_shifted_{age_group}_age_{sex}_sex.jpg", dpi=100)

In [ ]:
# Appendix
n_countries = len(countries)
n_cols = 4
n_rows = int(np.ceil(n_countries / n_cols))
age_string = {"high": "65+", "mid": "15-64", "adult": "15+"}

for group, age_df in combo.groupby(by=["age_group", "sex"]):
    age_group, sex = group
    if age_group != "adult" or sex != "b":
        continue
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(28,28))
    i = 0
    for country, df in age_df.groupby(by=["country"]):
        j = i % n_cols
        k = i // n_cols
        if df.loc[:, "location"].isnull().iloc[0]:
            continue
        axs[k, j].plot(df.date, df.centered_per, color="black", linewidth=2, label="mortality [%]")
        axs[k, j].plot(df.date, df.centered_per_low, color="black", linestyle='--')
        axs[k, j].plot(df.date, df.centered_per_high, color="black", linestyle='--')
        axs[k, j].set_title(df.loc[:, "location"].values[0], fontsize=20)
        axs[k, j].set_xlim([np.datetime64("2020-01-01"), np.datetime64("2023-01-01")])

        ax2 = axs[k, j].twinx()    
        any_dose = df.weekly_vaccinated + df.weekly_fully_vax + df.total_boosters_vax
        ax2.plot(df.date, any_dose, label="# doses", color="red", alpha=0.7)
        if j == n_cols - 1:
            ax2.set_ylabel("# Doses")
        if j == 0 and k % n_cols == 0:
            axs[k, j].set_ylabel(f"Excess mortality, age {age_string[age_group]} [% from baseline]")
        if k == 0:
            axs[k, j].legend(loc="upper left")
            ax2.legend(loc="upper right")
        i += 1


    fig.tight_layout()
    plt.savefig(f"../figures/all_countries_mortality_vs_vax_{age_group}_age_{sex}_sex.jpg", dpi=100)


In [ ]:
n_countries = len(countries)
n_cols = 4
n_rows = int(np.ceil(n_countries / n_cols))
age_string = {"high": "65+", "mid": "15-64", "adult": "15+"}

for group, age_df in combo.groupby(by=["age_group", "sex"]):
    age_group, sex = group
    if age_group != "adult" or sex != "b":
        continue
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(28,28))
    i = 0
    for country, df in age_df.groupby(by=["country"]):
        j = i % n_cols
        k = i // n_cols
        if df.loc[:, "location"].isnull().iloc[0]:
            continue
        axs[k, j].plot(df.date, df.centered_per, color="black", linewidth=2, label="mortality [%]")
        axs[k, j].plot(df.date, df.centered_per_low, color="black", linestyle='--')
        axs[k, j].plot(df.date, df.centered_per_high, color="black", linestyle='--')
        axs[k, j].set_title(df.loc[:, "location"].values[0], fontsize=20)
        axs[k, j].set_xlim([np.datetime64("2020-01-01"), np.datetime64("2023-01-01")])

        ax2 = axs[k, j].twinx()    
        any_dose = df.total_boosters_vax
        ax2.plot(df.date, any_dose, label="# doses", color="red", alpha=0.7)
        if j == n_cols - 1:
            ax2.set_ylabel("# Doses")
        if j == 0 and k % n_cols == 0:
            axs[k, j].set_ylabel(f"Excess mortality, age {age_string[age_group]} [% from baseline]")
        if k == 0:
            axs[k, j].legend(loc="upper left")
            ax2.legend(loc="upper right")
        i += 1


    fig.tight_layout()
    plt.savefig(f"../figures/all_countries_mortality_vs_booster_{age_group}_age_{sex}_sex.jpg", dpi=100)


In [ ]:
n_countries = len(countries)
n_cols = 4
n_rows = int(np.ceil(n_countries / n_cols))


age_string = {"high": "65+", "mid": "15-64", "adult": "15+"}

for age_group, age_df in combo.groupby(by=["age_group"]):
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(28,28))
    i = 0
    for gdf in age_df.groupby(by=["country"]):
        country, df = gdf
        df = df[(df["sex"] == "b")]
        df.index = pd.to_datetime(df.date)
        if df.loc[:, "location"].isnull().iloc[0]:
            continue
        
        j = i % n_cols
        k = i // n_cols
        axs[k, j].plot(df.date, df["fit.yearly"].values, color="gold", linewidth=2, label="trend")
        axs[k, j].plot(df.date, df["fit.yearly"].values + df["fit.weekly"].values, color="gold", linewidth=2)
        axs[k, j].plot(df.date, df["fit.yearly"].values + df["fit.weekly"].values + df["centered"].values, color="black", linewidth=1, label="mortality")
        axs[k, j].plot(df.date, df["fit.yearly"].values + df["fit.weekly"].values + df["centered_low"].values, color="black", linestyle='--', linewidth=0.5, label="95 confidence int.")
        axs[k, j].plot(df.date, df["fit.yearly"].values + df["fit.weekly"].values + df["centered_high"].values, color="black", linestyle='--', linewidth=0.5)
        axs[k, j].set_title(df.loc[:, "location"].values[0], fontsize=20)
    #     axs[k, j].set_xlim([np.datetime64("2017-01-01"), np.datetime64("2023-01-01")])

        if j == 0 and k % n_cols == 0:
            axs[k, j].set_ylabel(f"Mortality, age {age_string[age_group]} [# deaths/100k/week]")
        if k == 0:
            axs[k, j].legend(loc="upper left")
            ax2.legend(loc="upper right")
        i += 1


    fig.tight_layout()
    plt.savefig(f"../figures/all_countries_mortality_w_trend_b_gender_{age_group}_age.jpg", dpi=100)


In [ ]:
# ESP standard population
std_esp_pop = np.array((1000, 4000, 5500, 5500, 5500, 6000, 6000, 6500, 7000, 
            7000, 7000, 7000, 6500, 6000, 5500, 5000, 4000, 2500, 1500, 800, 200))
esp_group = np.array([0,1] + np.arange(5,95,5).tolist() + [200])
esp = pd.DataFrame({"age": esp_group, "n": std_esp_pop})
esp[esp.age>=65].n.sum()

In [ ]:
vax_orig = pd.read_csv("../data/owid_vaccinations.csv")
df = vax_orig[(vax_orig.iso_code == "SVK")]
df.index = pd.to_datetime(df.date)
df["2021-10":]

In [ ]:
# combo[(combo.country == "SVK") & (combo.sex == "b") & (age_group == "high")]
df = combo[(combo.country == "SVK") & (combo.sex == "b") & (combo.age_group=="adult")]
df.index = pd.to_datetime(df.date)

In [ ]:
mean_excess = pd.DataFrame(data=avg_mortality_percent.values(), index=avg_mortality_percent.keys(), columns=["excess_mortality"])
mean_excess.sort_values(by="excess_mortality")

In [ ]:
# NOT USED but maybe relevant - Plot shifted data 

n_countries = len(countries)
n_cols = 4
n_rows = int(np.ceil(n_countries / n_cols))
age_string = {"high": "65+", "mid": "15-64", "adult": "15+"}
from statsmodels.tools.sm_exceptions import InfeasibleTestError
for group, age_df in combo.groupby(by=["age_group", "sex"]):
    age_group, sex = group
    if age_group != "adult" or sex != "b":
        continue
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(28,28))
    i = 0
    for country, df in age_df.groupby(by=["country"]):
        j = i % n_cols
        k = i // n_cols
        if df.loc[:, "location"].isnull().iloc[0] or country == "TWN":
            continue
        axs[k, j].plot(df.date, df.centered_per, color="black", linewidth=2, label="mortality [%]")
        axs[k, j].plot(df.date, df.centered_per_low, color="black", linestyle='--')
        axs[k, j].plot(df.date, df.centered_per_high, color="black", linestyle='--')
        
        
        axs[k, j].set_xlim([np.datetime64("2020-01-01"), np.datetime64("2023-01-01")])

        ax2 = axs[k, j].twinx()    
        any_dose = df.total_boosters_vax
        ax2.plot(df.date, any_dose, label="# doses", color="red", alpha=0.7)
        # Calculate granger causality p-values
        try:
            if lags[country][0] >= 0 and not np.isnan(lags[country][1]):
                first_dose = np.arange(len(any_dose))[any_dose > 100][0]
                last_dose = np.arange(len(any_dose))[any_dose > 100][-1]
                h = granger_w_zero_lag(df.loc[:, ["centered_per", "total_boosters_vax"]].iloc[first_dose:last_dose,:], lags[country][0] + 1, verbose=False)
                p_val = h[lags[country][0] + 1][0]["ssr_ftest"][1]
                print(f"{country}: {p_val}")
                signif = p_val < 0.05
            else:
                signif = False
        except InfeasibleTestError as e:
            signif = False
        star = "*" if signif else ""
        title = f"{df.loc[:, 'location'].values[0]} - Shift: {lags[country][0]} week(s), R: {lags[country][1]}"
        axs[k, j].set_title(title, fontsize=20)
        if lags[country][0] == 0:
            ax2.plot(df.date, any_dose, label="shifted", color="blue", alpha=0.7)
        else:
            ax2.plot(df.date[lags[country][0]:], any_dose[:-lags[country][0]], label="shifted", color="blue", alpha=0.7)
        if j == n_cols - 1:
            ax2.set_ylabel("# Doses")
        if j == 0 and k % n_cols == 0:
            axs[k, j].set_ylabel(f"Excess mortality, age {age_string[age_group]} [% from baseline]")
        if k == 0:
            axs[k, j].legend(loc="upper left")
            ax2.legend(loc="upper right")
        i += 1


    fig.tight_layout()
#     plt.savefig(f"../figures/all_countries_mortality_vs_booster_{age_group}_age_{sex}_sex.jpg", dpi=100)
